# 第9章 LangChainで作るAIエージェント実践入門

## 設定

In [1]:
import json
import os
import time

from dotenv import load_dotenv
dotenv_path = "../.env"
load_dotenv(dotenv_path)

True

In [2]:
!pip install langchain-core==0.3.0
!pip install langchain-openai==0.2.0
!pip install langgraph==0.2.22
!pip install grandalf


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Q&Aエージェント

* ユーザからの質問に応じて、LLMに適切なロールを設定した上で質問に回答する
* 生成した回答が妥当かを判定し、妥当でない場合は回答を再生成する

### 回答ロールの定義

* 本アプリケーション固有のものなので、LangGraphとは直接関係がない

In [3]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください。"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

## stateの定義

* LangGraphで実装を進める際には、始めにデータ構造（state）を設計する

In [4]:
import operator
from typing import Annotated

from langchain_core.pydantic_v1 import BaseModel, Field

C:\Users\MikAn\Documents\Python\agent_book\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
class State(BaseModel):
    query: str = Field(
        ..., # 必須のフィールドであることを明示
        description="ユーザからの質問"
    )

    current_role: str = Field(
        default="",
        description="選択された回答ロール",
    )

    # フィールド更新時は通常値が上書きされる
    # Annotatedで明示的に更新方法を定義可能
    # この例ではListに値を追加するように定義
    messages: Annotated[list[str], operator.add] = Field(
        default=[],
        description="回答履歴",
    )

    current_judge: bool = Field(
        default=False,
        description="品質チェックの結果",
    )

    judgement_reason: str = Field(
        default="",
        description="品質チェックの判定理由",
    )

### nodeの定義

* nodeはstateを引数に受け取り、stateを更新するためのdictを返す関数として定義する

In [6]:
from langchain_openai import ChatOpenAI

In [7]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [8]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [9]:
# ユーザからの質問に応じて、LLMに割り当てる適切なロールを選択
def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "質問を分析し、最も適切な回答担当ロールを選択してください。\n"
                "回答は選択肢の番号（1、2、または3）のみ返してください。\n\n"
                "1. 一般知識エキスパート: 幅広い分野の一般的な質問に答える\n"
                "2. 生成AI製品エキスパート: 生成AIや関連製品、技術に関する専門的な質問に答える\n"
                "3. カウンセラー: 個人的な悩みや心理的な問題に対してサポートを提供する\n"
            ),
            (
                "human",
                "{query}",
            ),
        ]
    )

    chain = prompt | model | StrOutputParser()

    role_number = chain.invoke({"query": query})
    selected_role = ROLES[role_number.strip()]["name"]

    return {"current_role": selected_role}

In [10]:
# 割り当てられたロールになりきり、ユーザからの質問に回答する
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "あなたは{role}として回答してください。\n"
                "ユーザからの質問に基づいた適切な価値王を提供してください。\n\n"
                "役割の詳細:\n"
                "1. 一般知識エキスパート: 幅広い分野の一般的な質問に答える\n"
                "2. 生成AI製品エキスパート: 生成AIや関連製品、技術に関する専門的な質問に答える\n"
                "3. カウンセラー: 個人的な悩みや心理的な問題に対してサポートを提供する\n"
            ),
            (
                "human",
                "{query}",
            ),
        ]
    )

    chain = prompt | model | StrOutputParser()

    answer = chain.invoke({"role": role, "query": query})

    return {"messages": [answer]}

In [11]:
# 生成した回答が妥当かを判定した結果を格納するクラス
class Judgement(BaseModel):
    reason: str = Field(default="", description="判定理由")
    judge: bool = Field(default=False, description="判定結果")

# 生成した回答が妥当かを判定する
def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "ユーザから与えられる質問と回答を踏まえ、回答の品質をチェックしてください。\n"
                "問題がある場合は'False'、問題がない場合は'True'を回答してください。"
                "また、その判定理由も説明してください。"
            ),
            (
                "human",
                "質問: {query}\n"
                "回答: {answer}"
            )
        ]
    )

    chain = prompt | model.with_structured_output(Judgement) # 出力をJudgement型にParse

    result = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    } 

### グラフの定義

### グラフの定義

* stateをもとにworkflowを作成し、そこにnodeを追加し、edgeを張る
* 作成したグラフはコンパイルして実行可能な状態になる

In [12]:
from langgraph.graph import StateGraph

In [13]:
# StateGraphで空のworkflowを作成
workflow = StateGraph(State)

In [14]:
# workflowにnodeを追加
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [15]:
from langgraph.graph import END

In [16]:
# edgeを張る
workflow.set_entry_point("selection") # エントリーポイントを定義
workflow.add_edge("selection", "answering") # selection -> answering
workflow.add_edge("answering", "check") # answering -> check
workflow.add_conditional_edges( # check -> END or selectionの条件付きedge
    "check",
    lambda state: state.current_judge,
    {True: END, False:"selection"}
)

In [17]:
# コンパイル
compiled = workflow.compile()

In [18]:
# グラフをASCII文字で可視化
compiled.get_graph().print_ascii()

           +-----------+      
           | __start__ |      
           +-----------+      
                  *           
                  *           
                  *           
           +-----------+      
           | selection |      
           +-----------+      
           ***        ..      
          *             ..    
        **                ..  
+-----------+               . 
| answering |             ..  
+-----------+           ..    
           ***        ..      
              *     ..        
               **  .          
             +-------+        
             | check |        
             +-------+        
                  .           
                  .           
                  .           
            +---------+       
            | __end__ |       
            +---------+       


### エージェントの実行

実行例1

In [19]:
# エントリーポイントに入力するstateを作成
initial_state = State(query="生成AIについて教えてください")

In [20]:
# コンパイルされたグラフを実行
result = compiled.invoke(initial_state)
print(result["messages"][-1])

生成AI（Generative AI）とは、データを基に新しいコンテンツを生成する人工知能の一種です。テキスト、画像、音声、動画など、さまざまな形式のコンテンツを作成する能力があります。以下に、生成AIの主な特徴と応用例を紹介します。

### 特徴
1. **学習能力**: 大量のデータを学習し、そのパターンを理解することで新しいコンテンツを生成します。
2. **創造性**: 既存の情報を基に新しいアイデアや作品を生み出すことができます。
3. **多様性**: 同じ入力に対して異なる出力を生成することができ、さまざまなスタイルやトーンでコンテンツを作成できます。

### 主な応用例
1. **テキスト生成**: 自然言語処理を用いて、記事、ストーリー、詩などを自動生成します。例えば、ChatGPTやGPT-3などのモデルがあります。
2. **画像生成**: DALL-EやMidjourneyなどのツールを使用して、テキストから画像を生成することができます。
3. **音声生成**: 音声合成技術を用いて、リアルな音声を生成したり、特定の声を模倣したりすることが可能です。
4. **動画生成**: AIを用いて短い動画やアニメーションを生成する技術も進化しています。

### 利点と課題
- **利点**: コンテンツ制作の効率化、コスト削減、クリエイティブなアイデアの提供など。
- **課題**: 偽情報の生成、著作権の問題、倫理的な懸念などが存在します。

生成AIは、さまざまな業界での活用が期待されており、今後も技術の進化が続くでしょう。興味がある特定の分野や技術についてさらに詳しく知りたい場合は、お知らせください。


In [21]:
result

{'query': '生成AIについて教えてください',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AI（Generative AI）とは、データを基に新しいコンテンツを生成する人工知能の一種です。テキスト、画像、音声、動画など、さまざまな形式のコンテンツを作成する能力があります。以下に、生成AIの主な特徴と応用例を紹介します。\n\n### 特徴\n1. **学習能力**: 大量のデータを学習し、そのパターンを理解することで新しいコンテンツを生成します。\n2. **創造性**: 既存の情報を基に新しいアイデアや作品を生み出すことができます。\n3. **多様性**: 同じ入力に対して異なる出力を生成することができ、さまざまなスタイルやトーンでコンテンツを作成できます。\n\n### 主な応用例\n1. **テキスト生成**: 自然言語処理を用いて、記事、ストーリー、詩などを自動生成します。例えば、ChatGPTやGPT-3などのモデルがあります。\n2. **画像生成**: DALL-EやMidjourneyなどのツールを使用して、テキストから画像を生成することができます。\n3. **音声生成**: 音声合成技術を用いて、リアルな音声を生成したり、特定の声を模倣したりすることが可能です。\n4. **動画生成**: AIを用いて短い動画やアニメーションを生成する技術も進化しています。\n\n### 利点と課題\n- **利点**: コンテンツ制作の効率化、コスト削減、クリエイティブなアイデアの提供など。\n- **課題**: 偽情報の生成、著作権の問題、倫理的な懸念などが存在します。\n\n生成AIは、さまざまな業界での活用が期待されており、今後も技術の進化が続くでしょう。興味がある特定の分野や技術についてさらに詳しく知りたい場合は、お知らせください。'],
 'current_judge': True,
 'judgement_reason': '回答は生成AIについての基本的な定義、特徴、応用例、利点と課題を包括的に説明しており、情報が正確で関連性が高い。特に、具体的な例を挙げている点が良い。'}

実行例2

In [22]:
# エントリーポイントに入力するstateを作成
initial_state = State(query="親しい友人が失恋して落ち込んでいます、励ましてあげたいです")

In [23]:
# コンパイルされたグラフを実行
result = compiled.invoke(initial_state)
print(result["messages"][-1])

友人が失恋して落ち込んでいるとき、あなたのサポートがとても大切です。以下のような方法で励ましてあげると良いでしょう。

1. **話を聞く**: 友人がどんな気持ちを抱えているのか、じっくりと話を聞いてあげてください。感情を表現することは、癒しの第一歩です。

2. **共感を示す**: 友人の気持ちに共感し、「それは辛いよね」といった言葉をかけることで、理解していることを伝えましょう。

3. **ポジティブな面を見つける**: 失恋は辛い経験ですが、新しい出会いや成長の機会でもあります。「これから新しいことに挑戦できるかもしれないね」といった前向きな視点を提供してみてください。

4. **一緒に過ごす**: 友人が気分転換できるように、一緒に映画を見たり、散歩をしたりする時間を作ると良いでしょう。気分が少しでも楽になるかもしれません。

5. **時間をかけることを理解する**: 失恋から立ち直るには時間がかかることを理解し、焦らずにサポートし続ける姿勢を持ちましょう。

友人に寄り添い、支えてあげることで、少しでも心の負担を軽くしてあげられると良いですね。


In [24]:
result

{'query': '親しい友人が失恋して落ち込んでいます、励ましてあげたいです',
 'current_role': 'カウンセラー',
 'messages': ['友人が失恋して落ち込んでいるとき、あなたのサポートがとても大切です。以下のような方法で励ましてあげると良いでしょう。\n\n1. **話を聞く**: 友人がどんな気持ちを抱えているのか、じっくりと話を聞いてあげてください。感情を表現することは、癒しの第一歩です。\n\n2. **共感を示す**: 友人の気持ちに共感し、「それは辛いよね」といった言葉をかけることで、理解していることを伝えましょう。\n\n3. **ポジティブな面を見つける**: 失恋は辛い経験ですが、新しい出会いや成長の機会でもあります。「これから新しいことに挑戦できるかもしれないね」といった前向きな視点を提供してみてください。\n\n4. **一緒に過ごす**: 友人が気分転換できるように、一緒に映画を見たり、散歩をしたりする時間を作ると良いでしょう。気分が少しでも楽になるかもしれません。\n\n5. **時間をかけることを理解する**: 失恋から立ち直るには時間がかかることを理解し、焦らずにサポートし続ける姿勢を持ちましょう。\n\n友人に寄り添い、支えてあげることで、少しでも心の負担を軽くしてあげられると良いですね。'],
 'current_judge': True,
 'judgement_reason': '回答は友人を励ますための具体的な方法を提供しており、感情的なサポートの重要性を強調しています。また、共感やポジティブな視点を持つこと、友人と一緒に過ごすこと、時間をかけることの理解など、実践的で思いやりのあるアドバイスが含まれています。全体的に、回答は質が高く、質問に対して適切に応じています。'}

In [25]:
chain = prompt | model | output_parser
output = chain.invoke({"dish": "カレー"})
print(output)

NameError: name 'prompt' is not defined

**LCELでは各Runnable前後の型の整合性に注意！**  
* (dict) -> prompt -> (ChatPromptValue)
* (ChatPromptValue) -> model -> (AIMessage)
* (AIMessage) -> output_parser -> (string)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [ ]:
# 小文字を大文字化する関数
def upper(text: str) -> str:
    return text.upper()

In [ ]:
# 明示的に宣言せずとも、自動的にRunnable化する
chain = prompt | model | output_parser | upper
output = chain.invoke({"input": "Hello"})
print(output)

* (dict) -> prompt -> (ChatPromptValue)
* (ChatPromptValue) -> model -> (AIMessage)
* (AIMessage) -> output_parser -> (string)
* (string) -> upper -> (string)